Tokenization Notes
Common problems as a result of tokenization:
- LLM can't spell words
- LLM can't do super simple string processing (like reversing a string)
- LLM worse at non-English langauges
- LLM bad at arithmetic
- GPT-2 trouble coding python
- warning about trailing whitespace
- YAML over JSON with LLMs

tiktokenizer

Non english languages work less well in chatgpt.
Why? less training data, tokenizer is also trained with more english
Also training data - use much more tokens in foreign languages because of the way they get broken up. All the non-english text is stretched out from the perspective of the xformer. will create bigger tokens/larger groups in english

Coding
- python each space is being considered a token. if you use a lot of indentation with python, then each space gets used as context length. you're being too wasteful.

cl100k_base (GPT4 tokenizer)
- halved the tokens - why? because # of tokens in GPT4 tokenizer is roughly double (went from 50k to 100k)
- this is a "good thing" maybe, because this is much denser tokens. 
- roughly able to see twice as much text as context to predict next. 
- just increasing number of tokens does not necessarily better. softmax grows. there is a sweet spot where you have just right number of tokens in your vocabulary
- GPT4 tokenizer, 3 spaces get grouped together, 7 spaces got grouped 
- this "densifies" the text, which improved ability to code.

Why can't we just use integer for each character?
- the vocabulary would be very long
- the unicode standard keeps changing as well
- for these reasons we need something better.
- thus we turn to encodings (UTF-8, UTF-16, UTF-32)
- these encodings are ways we can take unicode text and convert them into byte strings
- UTF-8 takes every single code point and converts between 1 to 4 bytes (variable length)
- UTF-32 is fixed length not variable
- UTF-8 manifesto; UTF-8 is the only backwards compatible encoding to ascii
- if we use utf-8 naively, there are only 256 values, if we used this, all of our text would be stretched over very long sequences of bytes. we don't want to use raw bytes of utf-8 encoding. we want to use a variable vocabulary size which is tuned as a hyperparameter. 
- AK prefers if they could not do tokenization he would prefer it, but not proven out yet.

Byte-pair encoding
- this is a compression algorithm, can compress down until the to a vocabulary size is achieved
- suppose the data to be reocrded is
- aaabdaaabac - identify the pair that occurs most often, so you mint a new token, and replace every occurrence of this with the new token
- e.g.
- "aa" occurs most often, so replace with "Z"
"ZabdZabac"
Then we repeat with byte pair ab, replace it with "Y"
ZYdZYac
Y=ab
Z=aa
The only literal byte pair left occurs only once, and the encoding might stop here, or the process could continue with recursive byte pair encoding, replacing ZY with X
XdXac
X=ZY
Y=ab
Z=aa
after we go through this process, instead of having 11 tokens with vocab length of 4, we have vocab length of 7
we iteratively compress 
we start with byte seq of 256, we will iteratively start minting new tokens. we have an algorithm for getting up to a vocab size


In [72]:
# code
# unicode codepoints is a definnition of ~150K characters, what they look like
# and what integers represent these characters

ord("h") # 104
ord("y") # 121

[ord(x) for x in "hello world"]

list("hello world".encode("utf-16"))
# notice the 0, 101, 0 108 0 111 etc
# it's wasteful 
# same thing is observable with utf-32

text = "Unicode! UNICODE! The very name strikes fear and awe into the hearts of programmers."
text += """
The Tokenizer is a necessary and pervasive component of Large Language Models (LLMs), where it translates between strings and tokens (text chunks). Tokenizers are a completely separate stage of the LLM pipeline: they have their own training sets, training algorithms (Byte Pair Encoding), and after training implement two fundamental functions: encode() from strings to tokens, and decode() back from tokens to strings. In this lecture we build from scratch the Tokenizer used in the GPT series from OpenAI. In the process, we will see that a lot of weird behaviors and problems of LLMs actually trace back to tokenization. We'll go through a number of these issues, discuss why tokenization is at fault, and why someone out there ideally finds a way to delete this stage entirely. Chapters: 00:00:00 intro: Tokenization, GPT-2 paper, tokenization-related issues 00:05:50 tokenization by example in a Web UI (tiktokenizer) 00:14:56 strings in Python, Unicode code points 00:18:15 Unicode byte encodings, ASCII, UTF-8, UTF-16, UTF-32 00:22:47 daydreaming: deleting tokenization 00:23:50 Byte Pair Encoding (BPE) algorithm walkthrough 00:27:02 starting the implementation 00:28:35 counting consecutive pairs, finding most common pair 00:30:36 merging the most common pair 00:34:58 training the tokenizer: adding the while loop, compression ratio 00:39:20 tokenizer/LLM diagram: it is a completely separate stage 00:42:47 decoding tokens to strings 00:48:21 encoding strings to tokens 00:57:36 regex patterns to force splits across categories 01:11:38 tiktoken library intro, differences between GPT-2/GPT-4 regex 01:14:59 GPT-2 encoder.py released by OpenAI walkthrough 01:18:26 special tokens, tiktoken handling of, GPT-2/GPT-4 differences 01:25:28 minbpe exercise time! write your own GPT-4 tokenizer 01:28:42 sentencepiece library intro, used to train Llama 2 vocabulary 01:43:27 how to set vocabulary set? revisiting gpt.py transformer 01:48:11 training new tokens, example of prompt compression 01:49:58 multimodal [image, video, audio] tokenization with vector quantization 01:51:41 revisiting and explaining the quirks of LLM tokenization 02:10:20 final recommendations 02:12:50 ??? :) Exercises: - Advised flow: reference this document and try to implement the steps before I give away the partial solutions in the video. The full solutions if you're getting stuck are in the minbpe code https://github.com/karpathy/minbpe/blob/master/exercise.md Links: - Google colab for the video: https://colab.research.google.com/drive/1y0KnCFZvGVf_odSfcNAws6kcDD7HsI0L?usp=sharing - GitHub repo for the video: minBPE https://github.com/karpathy/minbpe - Playlist of the whole Zero to Hero series so far: https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ - our Discord channel: https://discord.gg/3zy8kqD9Cp - my Twitter: https://twitter.com/karpathy Supplementary links: - tiktokenizer https://tiktokenizer.vercel.app - tiktoken from OpenAI: https://github.com/openai/tiktoken - sentencepiece from Google https://github.com/google/sentencepiece
"""
tokens = text.encode("utf-8")
tokens = list(map(int, tokens)) # convert all bytes to list of integers in range 0..255 for convenience
tokens

print('---')
print(text)
print("length", len(text))
print('---')
print(tokens)
print("length:", len(tokens))

def get_sorted_pairs(token_list):
  # for ints in token_list, consider i and i+1 pair
  # as you iterate across list, for each i and i+1, add count to map
  # keep track of the max (or should there be a priority queue)
  i = 0
  pairs = {}
  while i < len(token_list) - 1:
    pair = (token_list[i], token_list[i+1])
    if pair in pairs:
      pairs[pair] += 1
    else:
      pairs[pair] = 1
    i += 1
  sorted_pairs = sorted(((v,k) for k,v in pairs.items()), reverse=True)
  return sorted_pairs # 4 occurrences of 101, 32

sorted_pairs = get_sorted_pairs(tokens)

chr(101), chr(32) # most common pair

# mint new token with 101, 32
# create a new token with 256, since values go from 0-255
# every time we see 101, 32 replace with 256

def replace_most_common(token_list, target, val):
  i = 0
  while i < len(token_list) - 1:
    if (token_list[i], token_list[i+1]) == target:
      token_list[i] = val
      token_list = token_list[:i+1] + token_list[i+2:]
    i += 1
  return token_list

# how to do the list manipulation
# list_a = [1,2,3,4,5]
# list_a
# i = 3
# list_a[i] = 6 
# list_a = list_a[:i+1] + list_a[i+2:] # works at pos 0, pos 1 to len-1, at pos = len-1, no match can be found so ok
# list_a

# sorted_pairs[0][1]
orig_tokens = list(tokens)

---
Unicode! UNICODE! The very name strikes fear and awe into the hearts of programmers.
The Tokenizer is a necessary and pervasive component of Large Language Models (LLMs), where it translates between strings and tokens (text chunks). Tokenizers are a completely separate stage of the LLM pipeline: they have their own training sets, training algorithms (Byte Pair Encoding), and after training implement two fundamental functions: encode() from strings to tokens, and decode() back from tokens to strings. In this lecture we build from scratch the Tokenizer used in the GPT series from OpenAI. In the process, we will see that a lot of weird behaviors and problems of LLMs actually trace back to tokenization. We'll go through a number of these issues, discuss why tokenization is at fault, and why someone out there ideally finds a way to delete this stage entirely. Chapters: 00:00:00 intro: Tokenization, GPT-2 paper, tokenization-related issues 00:05:50 tokenization by example in a Web UI (ti

In [73]:
vocab_count_desired = 275
new_token_count = 256
merges = {}
while new_token_count < vocab_count_desired:
  sorted_pairs = get_sorted_pairs(tokens)
  target = sorted_pairs[0][1] # always take the max
  merges[target] = new_token_count
  print("replacing ", target, " with val ", new_token_count)
  tokens = replace_most_common(tokens, target, new_token_count)
  new_token_count += 1

print(tokens)
print(len(tokens))
print(merges)

replacing  (32, 116)  with val  256
replacing  (101, 32)  with val  257
replacing  (105, 110)  with val  258
replacing  (101, 110)  with val  259
replacing  (115, 32)  with val  260
replacing  (101, 114)  with val  261
replacing  (99, 111)  with val  262
replacing  (258, 103)  with val  263
replacing  (256, 111)  with val  264
replacing  (107, 259)  with val  265
replacing  (256, 104)  with val  266
replacing  (97, 116)  with val  267
replacing  (32, 115)  with val  268
replacing  (111, 110)  with val  269
replacing  (97, 114)  with val  270
replacing  (44, 32)  with val  271
replacing  (105, 122)  with val  272
replacing  (265, 272)  with val  273
replacing  (114, 111)  with val  274
[85, 110, 105, 262, 100, 101, 33, 32, 85, 78, 73, 67, 79, 68, 69, 33, 32, 84, 104, 257, 118, 261, 121, 32, 110, 97, 109, 257, 115, 116, 114, 105, 107, 101, 260, 102, 101, 270, 32, 97, 110, 100, 32, 97, 119, 257, 258, 116, 111, 266, 257, 104, 101, 270, 116, 260, 111, 102, 32, 112, 274, 103, 114, 97, 109, 1

In [79]:
print("tokens length ", len(orig_tokens))
print("merged length ", len(tokens))
print(f"compression ratio: {len(orig_tokens) / len(tokens):.3f}X")

tokens length  3159
merged length  2534
compression ratio: 1.247X


Tokenizer is a completely separate part to the LLM
tokenizer has its own training set. You will train tokenizer, performing byte pair encoding to train the vocabulary of this tokenizer

Has a preprocessing stage, trained using BPE.
Once you have vocab, we can do encoding and decoding.
tokenizer is a translation layer between raw text and turn it into a token sequence, and can take a token sequence and convert it back into raw text.

how to do encoding and decoding. you choose different data sets which will influence your compression ratio. 
LLM
token sequence
tokenizer
raw text

In [110]:
# merges: {(32, 116): 256, (101, 32): 257
# vocab: 255: b'\xff', 256: b' t', 257: b'e ' ... 273: b'keniz', 274: b'ro'
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
  vocab[idx] = vocab[p0] + vocab[p1]
print(vocab)

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

def encode(text):
  # given raw text, encode using merges dictionary
  # should go backwards from the end to capture the longest token possible
  i = 0
  encoded = []
  # first need to convert all text into bytes
  bytes = text.encode("utf-8")
  while i < len(bytes):
    j = len(vocab) - 1
    while i < len(bytes) and j >= 0: # find longest match
      p = 0
      while p < len(vocab[j]) and (i+p) < len(bytes) and bytes[i+p] == vocab[j][p]:
        p += 1
        # text = "ro", vocab[274] = "ro", 
        # p=0
        # 'r' matches, p =1
        # p=1
        # 'o' matches, p =2
        # p=2 OOB
      if p == len(vocab[j]): # full match, replace string char with vocab index and jump forward p spaces
        # print("found match ", vocab[j])
        i += p
        encoded.append(j)
      else: # no match, decrement j
        j -= 1
  return encoded
  

text = "The Tokenizer is a necessary and pervasive component of Large Language Models (LLMs), "
text2 = decode(encode(text))
print(text2)
print(text2 == text)



{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

BPE in GPT tried to enforce non-merging across certain categories. Because "dog." "dog!" "dog,"
would maybe get merged as per BPE algo


In [117]:
import regex as re

#\p{L} is any character from any language
#\p{N} is any number
#[^\s\p{L}\p{N}]+ match punctuation
#\s+(?!\S) captures multiple spaces (using negative lookahead assertion)
#\s+ spaces
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Hello've world123 how's are      you!!!?"))

# this is saying we're never going to merge 'e ' 
# this chunks up the text to enforce that some merges
# don't actually happen. trying to not merge across
# letters, punctuation, etc

# why do to apostrophes? these are common contractions

['Hello', "'ve", ' world', '123', ' how', "'s", ' are', '     ', ' you', '!!!?']


In [118]:
# import tiktoken

ModuleNotFoundError: No module named 'tiktoken'

In GPT4 they changed the regex
special tokens and pattern change

load encoder.json
and open vocab.bpe

What's in these files?
in encoder.json is exactly equivalent to our vocab object
vocab[idx] = vocab[p0] + vocab[p1]
vocab.bpe is actually our merges

# saving and loading the two variables that were critical to us (encoder and decoder)

OPenAI also has a byte_encoder and a byte_decoder
they have a separate layer used serially with tokenizer byte_encode then encode and byte_decode then decode



Handling special tokens
# 256 raw byte tokens
# 50000 merges
but size is 50257
so what is that special otken?
called '<|endoftext|>'

used to delimit documents in the training set.
tokens in documents range from 0 ... 50256
we insert a special token in between tokens

we use this for the LLM that hte document has ended and what follows is unrelated to the document.

The LLM needs to learn this that what follows is uninformative to what came before.

# GPT4
has fill in the middle special tokens
special_tokens = {
    ENDOFTEXT: 100257,
    FIM_PREFIX: 100258,
    FIM_MIDDLE: 100259, 
    FIM_SUFFIX: 100260, 
    ENDOFPROMPT: 100276,
}

Sentencepiece

commonly used because it can efficienctly train and inference BPE tokenizers.
used by llama and mistral

under google/sentencepiece

One big difference between is that they think differently about the order

with tiktoken, we take code points, encode into UTF bytes, then merge bytes

with sentencepiece, takes code points, then starts merging those code points. BPE is running on level of code points vs BPE bytes

Code points will get mapped to unknown token ("unk") or will do a byte_fallback

sentencepiece been there for a while, a huge config and accumulated baggage
